# 신용카드 사기 검출

- ch7. Auto-encoder
    - undercomplete auto-encoder(과소완전 오토인코더)
    - overcomplete auto-encoder(과대완전 오토인코더)
    - sparse auto-encoder(희소 오토인코더)
    - denosing auto-encoder(노이즈 제거 오토인코더)
    - variational auto-encoder(변분 오토인코더)
- ch8. Hands-on Auto-encoder

In [26]:
!pip install --upgrade pip

Requirement already up-to-date: pip in /Users/csg/anaconda3/lib/python3.7/site-packages (20.2.3)


In [29]:
!pip install keras

In [30]:
!pip install --upgrade tensorflow

Requirement already up-to-date: tensorflow in /Users/csg/anaconda3/lib/python3.7/site-packages (2.3.0)


In [31]:
import tensorflow as tf
tf.__version__

'2.0.0'

In [32]:
## 라이브러리 불러오기
'''메인 라이브러리'''
import numpy as np
import pandas as pd
import os, time, re
import pickle, gzip

'''시각화 관련 라이브러리'''
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
import matplotlib as mpl

%matplotlib inline

'''데이터 준비 및 모델 평가 관련 라이브러리'''
from sklearn import preprocessing as pp
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import StratifiedKFold 
from sklearn.metrics import log_loss
from sklearn.metrics import precision_recall_curve, average_precision_score
from sklearn.metrics import roc_curve, auc, roc_auc_score

'''알고리즘 관련 라이브러리'''
import lightgbm as lgb

'''텐서플로 및 케라스 관련 라이브러리'''
import tensorflow as tf
import keras
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import Activation, Dense, Dropout
from keras.layers import BatchNormalization, Input, Lambda
from keras import regularizers
from keras.losses import mse, binary_crossentropy

ImportError: Keras requires TensorFlow 2.2 or higher. Install TensorFlow via `pip install tensorflow`

# 데이터 준비

In [ ]:
# 데이터 로드
current_path = os.getcwd()
file = os.path.sep.join(['', 'datasets', 'credit_card_data', 'credit_card.csv'])
data = pd.read_csv(current_path + file)

In [ ]:
dataX = data.copy().drop(['Class','Time'],axis=1)

In [ ]:
dataY = data['Class'].copy()

# 데이터 스케일

In [ ]:
featuresToScale = dataX.columns

In [ ]:
sX = pp.StandardScaler(copy=True, with_mean=True, with_std=True)
dataX.loc[:,featuresToScale] = sX.fit_transform(dataX[featuresToScale])

# 훈련 및 테스트 셋으로 분할 

In [ ]:
X_train, X_test, y_train, y_test = \
    train_test_split(dataX, dataY, test_size=0.33, \
                     random_state=2018, stratify=dataY)

In [ ]:
X_train_AE = X_train.copy()
X_test_AE = X_test.copy()

# 평가 함수(이상치 스코어 함수) 및 그래프 함수 정의

In [ ]:
def anomalyScores(originalDF, reducedDF):
    loss = np.sum((np.array(originalDF) - \
                   np.array(reducedDF))**2, axis=1)
    loss = pd.Series(data=loss,index=originalDF.index)
    loss = (loss-np.min(loss))/(np.max(loss)-np.min(loss))
    return loss

In [ ]:
def plotResults(trueLabels, anomalyScores, returnPreds = False):
    preds = pd.concat([trueLabels, anomalyScores], axis=1)
    preds.columns = ['trueLabel', 'anomalyScore']
    precision, recall, thresholds = \
        precision_recall_curve(preds['trueLabel'], \
                               preds['anomalyScore'])
    average_precision = average_precision_score( \
                        preds['trueLabel'], preds['anomalyScore'])
    
    plt.step(recall, precision, color='k', alpha=0.7, where='post')
    plt.fill_between(recall, precision, step='post', alpha=0.3, color='k')

    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.0])
    
    plt.title('Precision-Recall curve: Average Precision = \
        {0:0.2f}'.format(average_precision))

    fpr, tpr, thresholds = roc_curve(preds['trueLabel'], \
                                     preds['anomalyScore'])
    areaUnderROC = auc(fpr, tpr)

    plt.figure()
    plt.plot(fpr, tpr, color='r', lw=2, label='ROC curve')
    plt.plot([0, 1], [0, 1], color='k', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic: Area under the \
        curve = {0:0.2f}'.format(areaUnderROC))
    plt.legend(loc="lower right")
    plt.show()
    
    if returnPreds==True:
        return preds, average_precision  ### diff

# 모델 1
# 2-layers Complete AutoEncoder (linear activation function)

In [18]:
# 선형 활성화 함수로 구성된 2-계층 완전오토인코더

In [ ]:
# 신경망 API 호출
model = Sequential()

# 입력층에 선형 활성화 함수 적용
# 입력 계층과 동일한 29개 노드를 가진 은닉층 생성
model.add(Dense(units=29, activation='linear',input_dim=29))

# 은닉층에 선형 활성화 함수 적용
# 29개 노드를 가진 출력층 생성
model.add(Dense(units=29, activation='linear'))

In [ ]:
# 모델 컴파일
model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['accuracy'])

In [ ]:
# 모델 훈련
num_epochs = 10
batch_size = 32

history = model.fit(x=X_train_AE, y=X_train_AE,
                    epochs=num_epochs,
                    batch_size=batch_size,
                    shuffle=True,
                    validation_data=(X_train_AE, X_train_AE),
                    verbose=1)

In [ ]:
# 테스트 셋에 대한 평가
predictions = model.predict(X_test, verbose=1)
anomalyScoresAE = anomalyScores(X_test, predictions)
preds = plotResults(y_test, anomalyScoresAE, True)
model.reset_states()

In [ ]:
# 10번 실행 - 평균 정밀도의 평균 계산
test_scores = []
for i in range(0,10):
    # 신경망 API 호출
    model = Sequential()

    # 입력층에 선형 활성화 함수 적용
    # 입력층과 동일한 29개 노드를 가진 은닉층 생성
    model.add(Dense(units=29, activation='linear',input_dim=29))

    # 은닉층에 선형 활성화 함수 적용
    # 29개 노드를 가진 출력층 생성
    model.add(Dense(units=29, activation='linear'))

    # 모델 컴파일
    model.compile(optimizer='adam',
                  loss='mean_squared_error',
                  metrics=['accuracy'])

    # 모델 훈련
    num_epochs = 10
    batch_size = 32

    history = model.fit(x=X_train_AE, y=X_train_AE,
                        epochs=num_epochs,
                        batch_size=batch_size,
                        shuffle=True,
                        validation_data=(X_train_AE, X_train_AE),
                        verbose=1)

    # 테스트 셋에 대한 평가
    predictions = model.predict(X_test, verbose=1)
    anomalyScoresAE = anomalyScores(X_test, predictions)
    preds, avgPrecision = plotResults(y_test, anomalyScoresAE, True)
    test_scores.append(avgPrecision)
    model.reset_states()

print("Mean average precision over 10 runs: ", np.mean(test_scores))
test_scores

In [4]:
# 결과
print("Mean average precision over 10 runs: ", np.mean(test_scores))
print("Coefficient of variation over 10 runs: ", np.std(test_scores)/ \
                                                np.mean(test_scores))
test_scores

NameError: name 'test_scores' is not defined

# 모델 2
# 2-layers Under-complete AutoEncoder (linear activation function)

In [19]:
# 선형 활성화 함수를 가진 2-계층 과소완전 오토인코더(은닉층 20개 노드)

In [ ]:
# 10번 실행 - 평균 정밀도의 평균 계산
test_scores = []
for i in range(0,10):
    # 신경망 API 호출
    model = Sequential()

    # 입력층에 선형 활성화 함수 적용
    # 20개 노드를 가진 은닉층 생성
    model.add(Dense(units=20, activation='linear',input_dim=29))

    # 은닉층에 선형 활성화 함수 적용
    # 29개 노드를 가진 출력층 생성
    model.add(Dense(units=29, activation='linear'))

    # 모델 컴파일
    model.compile(optimizer='adam',
                  loss='mean_squared_error',
                  metrics=['accuracy'])

    # 모델 훈련
    num_epochs = 10
    batch_size = 32

    history = model.fit(x=X_train_AE, y=X_train_AE,
                        epochs=num_epochs,
                        batch_size=batch_size,
                        shuffle=True,
                        validation_data=(X_train_AE, X_train_AE),
                        verbose=1)

    # 테스트 셋에 대한 평가
    predictions = model.predict(X_test, verbose=1)
    anomalyScoresAE = anomalyScores(X_test, predictions)
    preds, avgPrecision = plotResults(y_test, anomalyScoresAE, True)
    test_scores.append(avgPrecision)
    model.reset_states()

print("Mean average precision over 10 runs: ", np.mean(test_scores))
test_scores

In [ ]:
# 결과
print("Mean average precision over 10 runs: ", np.mean(test_scores))
print("Coefficient of variation over 10 runs: ", np.std(test_scores)/ \
                                                np.mean(test_scores))
test_scores

# 모델 2 v2
# 2-layers Under-complete AutoEncoder (linear activation function)

In [20]:
# 선형 활성화 함수를 가진 2-계층 과소완전 오토인코더(은닉층 27개 노드)

In [ ]:
# 10번 실행 - 평균 정밀도의 평균 계산
test_scores = []
for i in range(0,10):
    # 신경망 API 호출
    model = Sequential()

    # 입력층에 선형 활성화 함수 적용
    # 27개 노드를 가진 은닉층 생성
    model.add(Dense(units=27, activation='linear',input_dim=29))

    # 은닉층에 선형 활성화 함수 적용
    # 29개 노드를 가진 출력층 생성
    model.add(Dense(units=29, activation='linear'))

    # 모델 컴파일
    model.compile(optimizer='adam',
                  loss='mean_squared_error',
                  metrics=['accuracy'])

    # 모델 훈련
    num_epochs = 10
    batch_size = 32

    history = model.fit(x=X_train_AE, y=X_train_AE,
                        epochs=num_epochs,
                        batch_size=batch_size,
                        shuffle=True,
                        validation_data=(X_train_AE, X_train_AE),
                        verbose=1)

    # 테스트 셋에 대한 평가
    predictions = model.predict(X_test, verbose=1)
    anomalyScoresAE = anomalyScores(X_test, predictions)
    preds, avgPrecision = plotResults(y_test, anomalyScoresAE, True)
    test_scores.append(avgPrecision)
    model.reset_states()

print("Mean average precision over 10 runs: ", np.mean(test_scores))
test_scores

In [ ]:
# 결과
print("Mean average precision over 10 runs: ", np.mean(test_scores))
print("Coefficient of variation over 10 runs: ", np.std(test_scores)/ \
                                                np.mean(test_scores))
test_scores

# 모델 3
# 3-layers Under-complete AutoEncoder (linear activation function)

In [22]:
# 선형 활성화 함수로 구성된 3-계층 과소완전 오토인코더(2개 은닉층에 각각 28개 노드, 27개 노드)

In [ ]:
# 10번 실행 - 평균 정밀도의 평균 계산
test_scores = []
for i in range(0,10):
    # 신경망 API 호출
    model = Sequential()

    # 입력층에 선형 활성화 함수 적용
    # 27개 노드를 가진 첫번째 은닉층 생성
    # 28개 노드를 가진 두번째 은닉층 생성
    model.add(Dense(units=28, activation='linear',input_dim=29))
    model.add(Dense(units=27, activation='linear'))

    # 두번째 은닉층에 선형 활성화 함수 적용
    # 29개 노드를 가진 출력층 생성
    model.add(Dense(units=29, activation='linear'))

    # 모델 컴파일
    model.compile(optimizer='adam',
                  loss='mean_squared_error',
                  metrics=['accuracy'])

    # 모델 훈련
    num_epochs = 10
    batch_size = 32

    history = model.fit(x=X_train_AE, y=X_train_AE,
                        epochs=num_epochs,
                        batch_size=batch_size,
                        shuffle=True,
                        validation_data=(X_train_AE, X_train_AE),
                        verbose=1)

    # 테스트 셋에 대한 평가
    predictions = model.predict(X_test, verbose=1)
    anomalyScoresAE = anomalyScores(X_test, predictions)
    preds, avgPrecision = plotResults(y_test, anomalyScoresAE, True)
    test_scores.append(avgPrecision)
    model.reset_states()

print("Mean average precision over 10 runs: ", np.mean(test_scores))
test_scores

In [ ]:
# 결과
print("Mean average precision over 10 runs: ", np.mean(test_scores))
print("Coefficient of variation over 10 runs: ", np.std(test_scores)/ \
                                                np.mean(test_scores))
test_scores

# 모델 4
# 4-layers Under-complete AutoEncoder (ReLu activation funtion)

In [ ]:
# ReLu 활성화 함수를 가진 4-계층 과소완전 오토인코더(4개의 은닉층에 각각 27개 노드,22개 노드,27개 노드,29개 노드)

In [ ]:
# 10번 실행 - 평균 정밀도의 평균 계산
test_scores = []
for i in range(0,10):
    # 신경망 API 호출
    model = Sequential()

    # 전체적으로 ReLu 활성화 함수 적용
    # 27개 노드를 가진 첫번째 은닉층 생성
    # 22개 노드를 가진 두번째 은닉층 생성
    model.add(Dense(units=27, activation='relu',input_dim=29))
    model.add(Dense(units=22, activation='relu'))

    # 전체적으로 ReLu 활성화 함수 적용
    # 27개 노드를 가진 세번째 은닉층 생성
    # 29개 노드를 가진 네번째 은닉층 생성
    model.add(Dense(units=27, activation='relu'))
    model.add(Dense(units=29, activation='relu'))

    # 모델 컴파일
    model.compile(optimizer='adam',
                  loss='mean_squared_error',
                  metrics=['accuracy'])

    # 훈련 모델
    num_epochs = 10
    batch_size = 32

    history = model.fit(x=X_train_AE, y=X_train_AE,
                        epochs=num_epochs,
                        batch_size=batch_size,
                        shuffle=True,
                        validation_data=(X_train_AE, X_train_AE),
                        verbose=1)

    # 테스트 셋에 대한 평가
    predictions = model.predict(X_test, verbose=1)
    anomalyScoresAE = anomalyScores(X_test, predictions)
    preds, avgPrecision = plotResults(y_test, anomalyScoresAE, True)
    test_scores.append(avgPrecision)
    model.reset_states()

print("Mean average precision over 10 runs: ", np.mean(test_scores))
test_scores

In [ ]:
# 결과
print("Mean average precision over 10 runs: ", np.mean(test_scores))
print("Coefficient of variation over 10 runs: ", np.std(test_scores)/ \
                                                np.mean(test_scores))
test_scores

# 모델 5

# 2-layers Over-complete AutoEncoder (linear activation funtion)

In [ ]:
# 선형 활성화 함수를 가진 2-계층 과대완전 오토인코더(은닉층 40개 노드)

In [ ]:
# 10번 실행 - 평균 정밀도의 평균 계산
test_scores = []
for i in range(0,10):
    # 신경망 API 호출
    model = Sequential()

    # 전체적으로 선형 활성화 함수 적용
    # 40개 노드를 가진 은닉층 생성
    model.add(Dense(units=40, activation='linear',input_dim=29))

    # 29개 노드를 가진 출력층 생성
    model.add(Dense(units=29, activation='linear'))

    # 모델 컴파일
    model.compile(optimizer='adam',
                  loss='mean_squared_error',
                  metrics=['accuracy'])

    # 모델 훈련
    num_epochs = 10
    batch_size = 32

    history = model.fit(x=X_train_AE, y=X_train_AE,
                        epochs=num_epochs,
                        batch_size=batch_size,
                        shuffle=True,
                        validation_data=(X_train_AE, X_train_AE),
                        verbose=1)

    # 테스트 셋에 대한 평가
    predictions = model.predict(X_test, verbose=1)
    anomalyScoresAE = anomalyScores(X_test, predictions)
    preds, avgPrecision = plotResults(y_test, anomalyScoresAE, True)
    test_scores.append(avgPrecision)
    model.reset_states()

print("Mean average precision over 10 runs: ", np.mean(test_scores))
test_scores

In [ ]:
# 결과
print("Mean average precision over 10 runs: ", np.mean(test_scores))
print("Coefficient of variation over 10 runs: ", np.std(test_scores)/ \
                                                np.mean(test_scores))
test_scores

# 모델 6
# 2-layers Under-complete AutoEncoder (linear activation function, dropout)

In [ ]:
# 드롭아웃과 선형 활성화 함수를 가진 2-계층 과소완전 오토인코더(은닉층 40개 노드, 드룹아웃 비율: 10%)

In [ ]:
# 10번 실행 - 평균 정밀도의 평균 계산
test_scores = []
for i in range(0,10):
    # 신경망 API 호출
    model = Sequential()

    model.add(Dense(units=40, activation='linear',input_dim=29))
    model.add(Dropout(0.10))

    # 29개 노드를 가진 출력층 생성
    model.add(Dense(units=29, activation='linear'))

    # 모델 컴파일
    model.compile(optimizer='adam',
                  loss='mean_squared_error',
                  metrics=['accuracy'])

    # 모델 훈련
    num_epochs = 10
    batch_size = 32

    history = model.fit(x=X_train_AE, y=X_train_AE,
                        epochs=num_epochs,
                        batch_size=batch_size,
                        shuffle=True,
                        validation_data=(X_train_AE, X_train_AE),
                        verbose=1)

    # 테스트 셋에 대한 평가
    predictions = model.predict(X_test, verbose=1)
    anomalyScoresAE = anomalyScores(X_test, predictions)
    preds, avgPrecision = plotResults(y_test, anomalyScoresAE, True)
    test_scores.append(avgPrecision)
    model.reset_states()

print("Mean average precision over 10 runs: ", np.mean(test_scores))
test_scores

In [ ]:
# 결과
print("Mean average precision over 10 runs: ", np.mean(test_scores))
print("Coefficient of variation over 10 runs: ", np.std(test_scores)/ \
                                                np.mean(test_scores))
test_scores

# 모델 7
# 2-layers Sparse Over-complete AutoEncoder (linear activation function)

In [ ]:
# 선형 활성화 함수를 가진 2-계층 희소 과대완전 오토인코더(은닉층 40개 노드)

In [ ]:
# 10번 실행 - 평균 정밀도의 평균 계산
test_scores = []
for i in range(0,10):
    # 신경망 API 호출
    model = Sequential()

    model.add(Dense(units=40, activation='linear',  \
        activity_regularizer=regularizers.l1(10e-5), input_dim=29))

    # 29개 노드를 가진 출력층 생성
    model.add(Dense(units=29, activation='linear'))

    # 모델 컴파일
    model.compile(optimizer='adam',
                  loss='mean_squared_error',
                  metrics=['accuracy'])

    # 모델 훈련
    num_epochs = 10
    batch_size = 32

    history = model.fit(x=X_train_AE, y=X_train_AE,
                        epochs=num_epochs,
                        batch_size=batch_size,
                        shuffle=True,
                        validation_data=(X_train_AE, X_train_AE),
                        verbose=1)

    # 테스트 셋에 대한 평가
    predictions = model.predict(X_test, verbose=1)
    anomalyScoresAE = anomalyScores(X_test, predictions)
    preds, avgPrecision = plotResults(y_test, anomalyScoresAE, True)
    test_scores.append(avgPrecision)
    model.reset_states()

print("Mean average precision over 10 runs: ", np.mean(test_scores))
test_scores

In [ ]:
# 결과
print("Mean average precision over 10 runs: ", np.mean(test_scores))
print("Coefficient of variation over 10 runs: ", np.std(test_scores)/ \
                                                np.mean(test_scores))
test_scores

# 모델 8
# 2-layers Sparse Over-complete AutoEncoder (linear activation function, dropout)

In [23]:
# 드롭아웃과 선형 활성화 함수를 가진 2-계층 희소 과대완전 오토인코더(은닉층 40개 노드, 드롭아웃 비율: 5%)

In [ ]:
# 10번 실행 - 평균 정밀도의 평균 계산
test_scores = []
for i in range(0,10):
    # 신경망 API 호출
    model = Sequential()

    model.add(Dense(units=40, activation='linear',  \
        activity_regularizer=regularizers.l1(10e-5), input_dim=29))
    model.add(Dropout(0.05))

    # 29개 노드를 가진 출력층 생성
    model.add(Dense(units=29, activation='linear'))

    # 모델 컴파일
    model.compile(optimizer='adam',
                  loss='mean_squared_error',
                  metrics=['accuracy'])

    # 모델 훈련
    num_epochs = 10
    batch_size = 32

    history = model.fit(x=X_train_AE, y=X_train_AE,
                        epochs=num_epochs,
                        batch_size=batch_size,
                        shuffle=True,
                        validation_data=(X_train_AE, X_train_AE),
                        verbose=1)

    # 테스트 셋에 대한 평가
    predictions = model.predict(X_test, verbose=1)
    anomalyScoresAE = anomalyScores(X_test, predictions)
    preds, avgPrecision = plotResults(y_test, anomalyScoresAE, True)
    test_scores.append(avgPrecision)
    model.reset_states()

print("Mean average precision over 10 runs: ", np.mean(test_scores))
test_scores

In [ ]:
# 결과
print("Mean average precision over 10 runs: ", np.mean(test_scores))
print("Coefficient of variation over 10 runs: ", np.std(test_scores)/ \
                                                np.mean(test_scores))
test_scores

# 모델 9
# 2-layers Denoising Under-complete AutoEncoder (linear activation function)

In [ ]:
# 선형 활성화 함수를 가진 2-계층 노이즈 제거 과소완전 오토인코더(은닉층 27개 노드)

In [ ]:
# 10번 실행 - 평균 정밀도의 평균 계산
test_scores = []

noise_factor = 0.50
X_train_AE_noisy = X_train_AE.copy() + noise_factor * \
 np.random.normal(loc=0.0, scale=1.0, size=X_train_AE.shape)
X_test_AE_noisy = X_test_AE.copy() + noise_factor * \
 np.random.normal(loc=0.0, scale=1.0, size=X_test_AE.shape) 
    
for i in range(0,10):
    # 신경망 API 호출
    model = Sequential()

    # 선형 활성화 함수를 가진 27개 노드를 가진 은닉층 생성
    model.add(Dense(units=27, activation='linear', input_dim=29))

    # 29개 노드를 가진 출력층 생성
    model.add(Dense(units=29, activation='linear'))

    # 모델 컴파일
    model.compile(optimizer='adam',
                  loss='mean_squared_error',
                  metrics=['accuracy'])

    # 모델 훈련
    num_epochs = 10
    batch_size = 32

    history = model.fit(x=X_train_AE_noisy, y=X_train_AE_noisy,
                        epochs=num_epochs,
                        batch_size=batch_size,
                        shuffle=True,
                        validation_data=(X_train_AE, X_train_AE),
                        verbose=1)

    # 테스트 셋에 대한 평가
    predictions = model.predict(X_test_AE_noisy, verbose=1)
    anomalyScoresAE = anomalyScores(X_test, predictions)
    preds, avgPrecision = plotResults(y_test, anomalyScoresAE, True)
    test_scores.append(avgPrecision)
    model.reset_states()

print("Mean average precision over 10 runs: ", np.mean(test_scores))
test_scores

In [ ]:
# 결과
print("Mean average precision over 10 runs: ", np.mean(test_scores))
print("Coefficient of variation over 10 runs: ", np.std(test_scores)/ \
                                                np.mean(test_scores))
test_scores

# 모델 10
# 2-layers Denoising Over-complete AutoEncoder (linear activation function)

In [ ]:
# 선형 활성화 함수를 가진 2-계층 노이즈 제거 과대완전 오토인코더(은닉층 40개 노드, 희소성 정규화, 드롭아웃 비율 : 5%)

In [ ]:
# 10번 실행 - 평균 정밀도의 평균 계산
test_scores = []

noise_factor = 0.50
X_train_AE_noisy = X_train_AE.copy() + noise_factor * \
 np.random.normal(loc=0.0, scale=1.0, size=X_train_AE.shape)
X_test_AE_noisy = X_test_AE.copy() + noise_factor * \
 np.random.normal(loc=0.0, scale=1.0, size=X_test_AE.shape) 
    
for i in range(0,10):
    # 신경망 API 호출
    model = Sequential()

    # 선형 활성화 함수 가진 40개 노드를 가진 은닉층 생성
    model.add(Dense(units=40, activation='linear',  \
        activity_regularizer=regularizers.l1(10e-5), input_dim=29))
    model.add(Dropout(0.05))

    # 29개 노드를 가진 출력층 생성
    model.add(Dense(units=29, activation='linear'))

    # 모델 컴파일
    model.compile(optimizer='adam',
                  loss='mean_squared_error',
                  metrics=['accuracy'])

    # 모델 훈련
    num_epochs = 10
    batch_size = 32

    history = model.fit(x=X_train_AE_noisy, y=X_train_AE_noisy,
                        epochs=num_epochs,
                        batch_size=batch_size,
                        shuffle=True,
                        validation_data=(X_train_AE, X_train_AE),
                        verbose=1)

    # 테스트 셋에 대한 평가
    predictions = model.predict(X_test_AE_noisy, verbose=1)
    anomalyScoresAE = anomalyScores(X_test, predictions)
    preds, avgPrecision = plotResults(y_test, anomalyScoresAE, True)
    test_scores.append(avgPrecision)
    model.reset_states()

print("Mean average precision over 10 runs: ", np.mean(test_scores))
test_scores

In [ ]:
# 결과
print("Mean average precision over 10 runs: ", np.mean(test_scores))
print("Coefficient of variation over 10 runs: ", np.std(test_scores)/ \
                                                np.mean(test_scores))
test_scores

# 모델 11
# 2-layers Denoising Over-complete AutoEncoder (ReLU activation function)

In [ ]:
# ReLU 활성화 함수를 가진 2-계층 노이즈 제거 과대완전 오토인코더(은닉층 40개 노드, 희소성 정규화, 드롭아웃 비율 : 5%)

In [10]:
# 10번 실행 - 평균 정밀도의 평균 계산
test_scores = []

noise_factor = 0.50
X_train_AE_noisy = X_train_AE.copy() + noise_factor * \
 np.random.normal(loc=0.0, scale=1.0, size=X_train_AE.shape)
X_test_AE_noisy = X_test_AE.copy() + noise_factor * \
 np.random.normal(loc=0.0, scale=1.0, size=X_test_AE.shape) 
    
for i in range(0,10):
    # 신경망 API 호출
    model = Sequential()

    # 선형 활성화 함수를 가진 40개 노드를 가진 은닉층 생성
    model.add(Dense(units=40, activation='relu',  \
        activity_regularizer=regularizers.l1(10e-5), input_dim=29))
    model.add(Dropout(0.05))

    # 29개 노드를 가진 출력층 생성
    model.add(Dense(units=29, activation='linear'))

    # 모델 컴파일
    model.compile(optimizer='adam',
                  loss='mean_squared_error',
                  metrics=['accuracy'])

    # 모델 훈련
    num_epochs = 10
    batch_size = 32

    history = model.fit(x=X_train_AE_noisy, y=X_train_AE_noisy,
                        epochs=num_epochs,
                        batch_size=batch_size,
                        shuffle=True,
                        validation_data=(X_train_AE, X_train_AE),
                        verbose=1)

    # 테스트 셋에 대한 평가
    predictions = model.predict(X_test_AE_noisy, verbose=1)
    anomalyScoresAE = anomalyScores(X_test, predictions)
    preds, avgPrecision = plotResults(y_test, anomalyScoresAE, True)
    test_scores.append(avgPrecision)
    model.reset_states()

print("Mean average precision over 10 runs: ", np.mean(test_scores))
test_scores

NameError: name 'X_train_AE' is not defined

In [ ]:
# 결과
print("Mean average precision over 10 runs: ", np.mean(test_scores))
print("Coefficient of variation over 10 runs: ", np.std(test_scores)/ \
                                                np.mean(test_scores))
test_scores